# Application to find closest available velibs

### Import the needed Libs

In [38]:
import urllib.request
import json
import pandas as pd
import unidecode

### Load and preprocess the data

In [39]:
def get_fresh_data():

    with urllib.request.urlopen("https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json") as url:
        s_disp = json.loads(url.read().decode())

    with urllib.request.urlopen("https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json") as url:
        s_pos = json.loads(url.read().decode())
    
    return s_disp, s_pos

In [40]:
def create_data_frame(s_disp, s_pos):
    
    station_disp = pd.DataFrame.from_dict(s_disp.get('data').get('stations')).set_index('station_id')
    station_pos = pd.DataFrame.from_dict(s_pos.get('data').get('stations')).set_index('station_id')
    
    st = station_disp.merge(station_pos, on='station_id', how='left').drop(['numBikesAvailable',
                            'num_docks_available', 'numDocksAvailable','is_installed','is_renting','last_reported',
                            'capacity','stationCode_y', 'num_bikes_available','is_returning','rental_methods'], axis = 1)

    only_types = pd.DataFrame(st["num_bikes_available_types"].to_list(), columns=['mech','elec'], index=st.index)
    new = only_types.join(only_types["mech"].apply(pd.Series).join(only_types["elec"].apply(pd.Series)))
    fin = st.join(new).drop(['num_bikes_available_types','mech','elec', 'stationCode_x'], axis = 1)
    return fin



In [49]:
s_disp, s_pos = get_fresh_data()
p = create_data_frame(s_disp, s_pos)

p.head()

,name,lat,lon,mechanical,ebike
station_id,,,,,
213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,2,3
99950133,André Mazet - Saint-André des Arts,48.853756,2.339096,19,1
516709288,Charonne - Robert et Sonia Delauney,48.855908,2.392571,4,1
36255,Toudouze - Clauzel,48.879296,2.337360,0,2
37815204,Mairie du 12ème,48.840855,2.387555,20,2


## Find the starting coordinates

In [42]:
def get_stating_coord():
    lat, lon = 0, 0
    shit = input('input : ')
    unaccented_string = unidecode.unidecode(shit)
    address = '+'.join(unaccented_string.split())

    with urllib.request.urlopen(f"https://nominatim.openstreetmap.org/search?q={address}&format=json") as url:
        geo = json.loads(url.read().decode())
    try:
        lat = geo[0].get('lat') 
        lon = geo[0].get('lon')
    except:
        print('Couldn\'t find that address')
    
    return lat, lon

In [52]:
l1, l2 = get_stating_coord()

input : 7 rue cail


## Get the adress from the lat and lon

In [44]:
def get_address(lat, lon):
    with urllib.request.urlopen(f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json") as url:
        addr = json.loads(url.read().decode())
    return addr.get('display_name')

In [54]:
print(get_address(l1,l2))

7, Rue Cail, Quartier Saint-Vincent-de-Paul, Paris, Île-de-France, France métropolitaine, 75010, France


## Get the closest station

In [46]:
def find_closest_stations(df,lat,lon,num_station_print):
    t = df.iloc[(abs(p['lat']-float(lat))+abs(p['lon']-float(lon))).argsort()[:num_station_print]]
    return t

In [53]:
num_stations_print = 5
closest = find_closest_stations(p, l1,l2,num_stations_print)
closest

,name,lat,lon,mechanical,ebike
station_id,,,,,
27414902,Louis Blanc - La Chapelle,48.884009,2.359934,1,1
125542925,Louis Blanc - Aqueduc,48.882030,2.363562,2,6
653065577,Gare du Nord - Demarquay,48.881154,2.358411,4,0
1599921040,Jessaint - Marx Dormoy,48.884845,2.358930,1,5
49085250,Departement - Pajol,48.886675,2.361361,2,6


## References

Position data provided by : https://nominatim.org <br>
Vélib data provided by : https://www.velib-metropole.fr/donnees-open-data-gbfs-du-service-velib-metropole